# Prepare train data

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

train_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/train.json"
df_train = pd.read_json(train_data_path, lines=True)
df_train.head()

Mounted at /content/drive


,clusters,sentences,ner,relations,doc_key
0,"[[[17, 20], [23, 23]]]","[[English, is, shown, to, be, trans-context-fr...","[[[0, 0, Material], [10, 10, OtherScientificTe...","[[], [[29, 29, 31, 32, CONJUNCTION], [48, 49, ...",J87-1003
1,"[[[62, 64], [90, 91], [96, 98], [112, 114]], [...","[[In, this, paper, ,, a, novel, method, to, le...","[[[6, 6, Method], [10, 12, OtherScientificTerm...","[[[6, 6, 10, 12, USED-FOR], [10, 12, 14, 16, U...",CVPR_2003_18_abs
2,"[[[154, 154], [214, 214]], [[40, 44], [85, 85]...","[[In, this, paper, ,, we, present, a, digital,...","[[[7, 13, Method], [15, 21, Method], [23, 25, ...","[[[7, 13, 15, 21, USED-FOR], [15, 21, 23, 25, ...",INTERSPEECH_2013_31_abs
3,"[[[3, 3], [110, 110]]]","[[We, propose, a, method, that, automatically,...","[[[3, 3, Generic], [7, 7, OtherScientificTerm]...","[[[3, 3, 7, 7, USED-FOR], [7, 7, 20, 23, USED-...",I05-5008
4,"[[[35, 35], [69, 69]]]","[[Graph, unification, remains, the, most, expe...","[[[0, 1, Task], [8, 10, Task]], [[16, 17, Meth...","[[[0, 1, 8, 10, PART-OF]], [[16, 17, 22, 23, P...",C92-2068


In [2]:
processed_sentence_list = []
processed_entity_list = []
for index, row in df_train.iterrows():
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      # entity_list = entity_list.pop()
      for i in entity_list:
        if type(i) == list and len(i) == 3:
          (start, end, label) = i
          phrase = " ".join(abstract_list[start:end+1])
          phrase = phrase
          if phrase in text:
            updated_entity_list.append((phrase, label))
      entity = [tuple(sublist)for sublist in updated_entity_list]
      processed_sentence_list.append(text)

      processed_entity_list.append(entity)

print(processed_sentence_list[0])
print(processed_entity_list[0])

English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement .
[('English', 'Material'), ('coordinations', 'OtherScientificTerm'), ('strictly syntactic cross-serial agreement', 'OtherScientificTerm')]


In [3]:
task_description = "Please extract named entities from the description and label it from {[Task], [Method], [Material], [Matric], [Generic], [OtherScientificTerm]}"

f = open("train.txt", "a")
for i in range(len(processed_sentence_list)):
  solution = ""
  for (word, label) in processed_entity_list[i]:
    solution += word
    solution += "(" + label + ")" + ", "
  prompt = task_description + "\n" + "Description: " + processed_sentence_list[i] + "\n" + "Extract entities: " + solution 
  f.write(prompt)
  f.write("\n")
  f.write(" ")
  f.write("\n")
f.close()

# Prepare valid data

In [4]:
valid_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/dev.json"
df_valid = pd.read_json(valid_data_path, lines=True)
df_valid.head()

,clusters,sentences,ner,relations,doc_key
0,"[[[6, 17], [32, 32]], [[4, 4], [55, 55], [91, ...","[[This, paper, presents, an, algorithm, for, c...","[[[4, 4, Generic], [6, 17, Task], [20, 21, Mat...","[[[4, 4, 6, 17, USED-FOR], [20, 21, 4, 4, USED...",ICCV_2003_158_abs
1,"[[[90, 91], [107, 107]]]","[[Past, work, of, generating, referring, expre...","[[[4, 5, OtherScientificTerm], [12, 13, OtherS...","[[], [], [], [], [], []]",C04-1096
2,"[[[32, 32], [44, 44]], [[1, 2], [11, 11]], [[1...","[[An, entity-oriented, approach, to, restricte...","[[[1, 2, Method], [4, 5, Task]], [[11, 11, Gen...","[[[1, 2, 4, 5, USED-FOR]], [], [[32, 32, 37, 3...",P84-1047
3,"[[[6, 11], [21, 21], [53, 53]], [[15, 16], [69...","[[This, paper, summarizes, the, formalism, of,...","[[[4, 11, Task], [6, 11, OtherScientificTerm],...","[[[15, 16, 19, 19, USED-FOR]], [], [[61, 62, 5...",C88-1066
4,"[[[34, 36], [99, 101]], [[3, 5], [27, 27], [48...","[[We, present, a, text, mining, method, for, f...","[[[3, 5, Method], [8, 9, OtherScientificTerm],...","[[[3, 5, 8, 9, USED-FOR], [13, 14, 3, 5, USED-...",C04-1116


In [5]:
processed_sentence_list = []
processed_entity_list = []
for index, row in df_valid.iterrows():
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      # entity_list = entity_list.pop()
      for i in entity_list:
        if type(i) == list and len(i) == 3:
          (start, end, label) = i
          phrase = " ".join(abstract_list[start:end+1])
          phrase = phrase
          if phrase in text:
            updated_entity_list.append((phrase, label))
      entity = [tuple(sublist)for sublist in updated_entity_list]
      processed_sentence_list.append(text)

      processed_entity_list.append(entity)

print(processed_sentence_list[0])
print(processed_entity_list[0])

This paper presents an algorithm for computing optical flow , shape , motion , lighting , and albedo from an image sequence of a rigidly-moving Lambertian object under distant illumination .
[('algorithm', 'Generic'), ('computing optical flow , shape , motion , lighting , and albedo', 'Task'), ('image sequence', 'Material'), ('rigidly-moving Lambertian object', 'Material'), ('distant illumination', 'OtherScientificTerm')]


In [6]:
task_description = "Please extract named entities from the description and label it from {[Task], [Method], [Material], [Matric], [Generic], [OtherScientificTerm]}"

f = open("dev.txt", "a")
for i in range(len(processed_sentence_list)):
  solution = ""
  for (word, label) in processed_entity_list[i]:
    solution += word
    solution += "(" + label + ")" + ", "
  prompt = task_description + "\n" + "Description: " + processed_sentence_list[i] + "\n" + "Extract entities: " + solution 
  f.write(prompt)
  f.write("\n")
  f.write(" ")
  f.write("\n")
f.close()

# GPT-2 structure (Freeze all layers except last one)

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.8 MB/s eta 0:00:00


In [8]:

from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,
          valid_file_path,
          model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  valid_dataset = load_dataset(valid_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  # Freeze all layers except last one:
  for parameter in model.parameters():
      parameter.requires_grad = False

  for i, m in enumerate(model.transformer.h):        
      #Only un-freeze the last n transformer blocks
      if i+1 > 12 - 1:
          for parameter in m.parameters():
              parameter.requires_grad = True 

  for parameter in model.transformer.ln_f.parameters():        
      parameter.requires_grad = True

  for parameter in model.lm_head.parameters():        
      parameter.requires_grad = True

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
          eval_steps=1000,
          evaluation_strategy="steps"
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=valid_dataset,
  )
      
  trainer.train()
  trainer.save_model()

# Fine-tune

In [9]:
train_file_path = "/content/train.txt"
valid_file_path = "/content/dev.txt"
model_name = 'gpt2'
output_dir = '/content/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 300
save_steps = 3000

In [10]:
train(
    train_file_path=train_file_path,
    valid_file_path=valid_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,1.570900,1.605618
2000,1.360600,1.601609
3000,1.208200,1.641370
4000,1.082000,1.712531
5000,0.960300,1.799970
6000,0.853000,1.897700
7000,0.761900,1.992905
8000,0.682100,2.071328
9000,0.612600,2.146875
10000,0.553400,2.216952


# Inference test

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/drive/MyDrive/ECSE552_project/gpt2_150epoch_sciERC/gpt2-150epoch-sciERC"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return (tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [12]:
sequence = """
Extract keyword from: Stable Diffusion is a deep learning, text-to-image model released by startup StabilityAI in 2022. It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.
"""
max_len = 200
text = generate_text(sequence, max_len) 
print(text)


Extract keyword from: Stable Diffusion is a deep learning, text-to-image model released by startup StabilityAI in 2022. It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.
Extract entities: Stable Diffusion(Method), deep learning(Method), text-to-image model(Method), it(Generic), tasks(Generic), inpainting(Task), outpainting(Task), 
 
Please extract named entities from the description and label it from {[Task], [Method], [Material], [Matric], [Generic], [OtherScientificTerm]}
Description: We show that it is feasible to create a language model that is both robust and flexible and also that it can also be applied to tasks involving image translations, text generation, machine translation, machine


# Inference (sciERC)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
test_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/test.json"
df_test = pd.read_json(test_data_path, lines=True)

In [9]:
processed_sentence_list = []
processed_entity_list = []
for index, row in df_test.iterrows():
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      # entity_list = entity_list.pop()
      for i in entity_list:
        if type(i) == list and len(i) == 3:
          (start, end, label) = i
          phrase = " ".join(abstract_list[start:end+1])
          phrase = phrase
          if phrase in text:
            updated_entity_list.append((phrase, label))
      entity = [tuple(sublist)for sublist in updated_entity_list]
      processed_sentence_list.append(text)

      processed_entity_list.append(entity)

print(len(processed_sentence_list))
print(len(processed_entity_list))

529
529


In [16]:
processed_sentence_list[0]

'Recognition of proper nouns in Japanese text has been studied as a part of the more general problem of morphological analysis in Japanese text processing -LRB- -LSB- 1 -RSB- -LSB- 2 -RSB- -RRB- .'

In [17]:
processed_entity_list[0]

[('Recognition of proper nouns', 'Task'),
 ('proper nouns', 'OtherScientificTerm'),
 ('Japanese text', 'Material'),
 ('morphological analysis', 'Task'),
 ('Japanese text processing', 'Task')]

In [18]:
from tqdm import tqdm

task_description = "Please extract named entities from the description and label it from {[Task], [Method], [Material], [Matric], [Generic], [OtherScientificTerm]}"

model_path = "/content/result"
model = load_model(model_path)
tokenizer = load_tokenizer(model_path)

generation_list = []
for index in tqdm(range(len(processed_sentence_list))):
  sentence = processed_sentence_list[index]
  prompt = task_description + "\n" + "Description:" + sentence + "\n" + "Extracted Keyword:"
  ids = tokenizer.encode(f'{prompt}', return_tensors='pt')
  final_outputs = model.generate(
      ids,
      do_sample=True,
      max_length=100,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95,
  )
  text = (tokenizer.decode(final_outputs[0], skip_special_tokens=True))
  generated_text = text.partition("Extracted Keyword:")[2].partition("\n")[0]
  generation_list.append(generated_text)


 62%|██████▏   | 329/529 [03:59<02:28,  1.35it/s]Input length of input_ids is 108, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
100%|██████████| 529/529 [06:18<00:00,  1.40it/s]


In [19]:
percentile_list = pd.DataFrame(
    {'sentence': processed_sentence_list,
     'entity': processed_entity_list,
     'generated text': generation_list
    })

percentile_list.to_csv("gpt2_scierc_50epoch.csv", index = False)

# Move fine-tuned model to MyDrive

In [ ]:
%cd /content/result
%cp -av /content/result/gpt2-150epoch-sciERC /content/drive/MyDrive/ECSE552_project/gpt2_150epoch_sciERC

# Evaluation

In [1]:
!git clone https://github.com/davidsbatista/NER-Evaluation.git
!pip3 install scikit-learn
!pip3 install nltk
!pip3 install sklearn_crfsuite

Cloning into 'NER-Evaluation'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 234 (delta 0), reused 2 (delta 0), pack-reused 230
Receiving objects: 100% (234/234), 82.22 KiB | 9.14 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.7 MB/s eta 0:00:00


In [3]:
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict
import pandas as pd

from sklearn_crfsuite.metrics import flat_classification_report

from NER_Evaluation.ner_evaluation.ner_eval import collect_named_entities
from NER_Evaluation.ner_evaluation.ner_eval import compute_metrics
from NER_Evaluation.ner_evaluation.ner_eval import compute_precision_recall_wrapper
from NER_Evaluation.ner_evaluation.ner_eval import namedtuple
from NER_Evaluation.ner_evaluation.ner_eval import compute_precision_recall
from NER_Evaluation.ner_evaluation.ner_eval import Evaluator

In [4]:
data = pd.read_csv("/content/gpt2_scierc_300epoch.csv")
data

,sentence,entity,generated text
0,Recognition of proper nouns in Japanese text h...,"[('Recognition of proper nouns', 'Task'), ('pr...",recognition
1,It has also been studied in the framework of J...,"[('It', 'Generic'), ('Japanese information ext...","It(Generic), Japanese information extraction(..."
2,Our approach to the Multi-lingual Evaluation T...,"[('approach', 'Generic'), ('Multi-lingual Eval...","approach(Generic), Japanese(Material), automa..."
3,Our morphological analyzer has done all the ne...,"[('morphological analyzer', 'Method'), ('recog...","morphological analyzer(Generic), Japanese tex..."
4,The analyzer is called `` Amorph '' .,"[('analyzer', 'Generic'), (""`` Amorph ''"", 'Me...","analyzer(Generic),"
...,...,...,...
524,Preliminary modeling and recognition results a...,"[('recognition', 'Task')]","model-based recognition(Task), method(Generic),"
525,Fast algorithms for nearest neighbor -LRB- NN ...,"[('Fast algorithms', 'Generic'), ('nearest nei...","fast algorithms(Method), fast local features(..."
526,Here we develop an approach for 1 distance tha...,"[('approach', 'Generic'), ('1 distance', 'Othe...","approach(Generic), embedding(OtherScientificT..."
527,We show how this can efficiently be combined w...,"[('this', 'Generic'), ('random-projection base...","Task(Generic), method(Generic), detecting 3D o..."


In [22]:
predicted_entities = []
correct_entities = []
Entity = namedtuple("Entity", "e_type start_offset end_offset")

def find_index_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1

for index, row in data.iterrows():

  sentence = row["sentence"]
  
  # sublist created for each sentence
  sub_predicted_entities = []
  sub_correct_entities = []

  # process true labeled entity list
  for entity in processed_entity_list[index]:
    (text, label) = entity
    start_index, end_index = find_index_sub_list(text.split(), sentence.split())
    sub_correct_entities.append(Entity(label, start_index, end_index))
  
  # process predicted entity list
  solution_list = row['entity']
  try:
    generated_list = row['generated text'].split(", ")

    for i in generated_list:
      if "(" in i and ")" in i:
        entity = i.partition("(")[0]
        label = i.partition("(")[2].partition(")")[0]
        try:
          start_index, end_index = find_index_sub_list(text.split(), sentence.split())
          sub_predicted_entities.append(Entity(label, start_index, end_index))
        except:
          continue
  except:
    continue

  predicted_entities.append(sub_predicted_entities)
  correct_entities.append(sub_correct_entities)

print(len(predicted_entities))
print(len(correct_entities))

527
527


In [23]:
tags = ['Task', "Method", "Material", "EvaluationMatric", "OtherScientificTerm", "Generic"]

metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
                   'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0}

# overall results
results = {'strict': deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results),
           'partial':deepcopy(metrics_results),
           'exact':deepcopy(metrics_results)
          }

for true_ents, pred_ents in zip(correct_entities, predicted_entities):

  # compute results for one message
  tmp_results, tmp_agg_results = compute_metrics(true_ents, pred_ents, tags)

  # aggregate overall results
  for eval_schema in results.keys():
    for metric in metrics_results.keys():
        results[eval_schema][metric] += tmp_results[eval_schema][metric]
  
  # Calculate global precision and recall
  results = compute_precision_recall_wrapper(results)

In [24]:
results

{'ent_type': {'correct': 310,
  'incorrect': 578,
  'partial': 0,
  'missed': 1157,
  'spurious': 41,
  'possible': 2045,
  'actual': 929,
  'precision': 0.333692142088267,
  'recall': 0.15158924205378974},
 'partial': {'correct': 880,
  'incorrect': 0,
  'partial': 8,
  'missed': 1157,
  'spurious': 41,
  'possible': 2045,
  'actual': 929,
  'precision': 0.9515608180839612,
  'recall': 0.4322738386308069},
 'strict': {'correct': 306,
  'incorrect': 582,
  'partial': 0,
  'missed': 1157,
  'spurious': 41,
  'possible': 2045,
  'actual': 929,
  'precision': 0.3293864370290635,
  'recall': 0.14963325183374082},
 'exact': {'correct': 880,
  'incorrect': 8,
  'partial': 0,
  'missed': 1157,
  'spurious': 41,
  'possible': 2045,
  'actual': 929,
  'precision': 0.9472551130247578,
  'recall': 0.43031784841075793}}